# Active-set for QP method

### Implemented
- Basic active-set QP algorithm
- Starting the QP by finding $x_0$ with the Simplex method
- Automatic standard form conversion (less-than inequalities to greater-than inequalities)
- Factoring the full KKT system
    - using Schur method
    - using the NULL-space method to later be able to perform factorization updates as described in Section 16.5 "UPDATING FACTORIZATIONS". Unfortunately, we did not succeed in updating the factorizations.
- Specification of the active-set method for convex QP, i.e., implemented the advanced version of the phase one problem where we can input an initial guess for a feasible point $x_0$.

<i>Group 08<br>
Participants information in alphabetical order</i>
<table style="width:100%">
  <
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Alexander</td>
    <td style = "text-align: left">Temper</td>
    <td style = "text-align: left">K11905007</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Bernhard Michael</td>
    <td style = "text-align: left">Voggenberger</td>
    <td style = "text-align: left">K11907093</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Ganhör</td>
    <td style = "text-align: left">K11911652</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Christoph</td>
    <td style = "text-align: left">Koller</td>
    <td style = "text-align: left">K11910272</td>
    </tr>
  <tr>
    <td style = "text-align: left">5</td>
    <td style = "text-align: left">Franziska</td>
    <td style = "text-align: left">Denk</td>
    <td style = "text-align: left">K11904292</td>
  </tr>
  <tr>
    <td style = "text-align: left">6</td>
    <td style = "text-align: left">Lukas</td>
    <td style = "text-align: left">Gattermayr</td>
    <td style = "text-align: left">K11911639</td>
  </tr>
  <tr>
    <td style = "text-align: left">7</td>
    <td style = "text-align: left">Nathanael</td>
    <td style = "text-align: left">Harmetzky</td>
    <td style = "text-align: left">K11916566</td>
  </tr>
  <tr>
    <td style = "text-align: left">8</td>
    <td style = "text-align: left">Raphael-Pascal</td>
    <td style = "text-align: left">Endstrasser</td>
    <td style = "text-align: left">K11907909</td>
  </tr>
  <tr>
    <td style = "text-align: left">9</td>
    <td style = "text-align: left">Tobias</td>
    <td style = "text-align: left">Stierberger</td>
    <td style = "text-align: left">K11907869</td>
  </tr>
  <tr>
    <td style = "text-align: left">10</td>
    <td style = "text-align: left">***</td>
    <td style = "text-align: left">***</td>
    <td style = "text-align: left">***</td>
  </tr>
</table>


In [1]:
import numpy as np
import scipy 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from typing import Callable, Tuple
from dataclasses import dataclass

In [2]:
# As we are using additional .py files, enable their reloading without restarting the kernel
%load_ext autoreload
%autoreload 2

from shared.printout import final_printout
from quadratic.base import minimize_quadratic_problem
from quadratic.quadratic_problem import QuadraticProblem
from quadratic.problems import create_another_example, create_example_16_4, create_exercise_example_16_1
from shared.constraints import LinearCallable, LinearConstraint, EquationType

# hide warnings about future changes in libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Problem 1
Taken from the book (exercise example 16.1).

\begin{align*}
\min 2x_1 + 3x_2 + 4x_1^2 + 2x_1x_2 + x_2^2 &\text{ subject to} \\
x_1 - x_2 &\geq 0 \\
x_1 + x_2 &\leq 4 \\
x_1 &\leq 3
\end{align*}

In [3]:
problem = create_exercise_example_16_1()
x_minimizer, iter_count = minimize_quadratic_problem(problem)
final_printout(problem.x0, problem.solution, x_minimizer, iter_count, problem.calc_f_at, problem.calc_gradient_at)

Initial x is :		None
Optimal x is :		[ 0.16666667 -1.66666667]
Approximated x is :	[ 0.16666667 -1.66666667]
Is close verification: 	[ True  True]

Function value in optimal point:	-2.3333333333333335
Function value in approximated point:   -2.3333333333333335
Is close verification:	True

Gradient approximation in optimal point is:
[0. 0.]

Gradient approximation in approximated point is:
[0.00000000e+00 1.49011612e-08]

Is close verification:
[ True  True]

Number of iterations required: 2


# Problem 2
Taken from the book (non-exercise example 16.4).

\begin{align*}
\min (x_1-1)^2 + (x_2-2.5)^2 &\text{ subject to} \\
x_1 - 2x_2 + 2 &\geq 0 \\
-x_1 - x_2 + 6 &\geq 0 \\
-x_1 + 2x_2 + 2 &\geq 0 \\
x_1 &\geq 0 \\
x_2 &\geq 0
\end{align*}

In [4]:
problem = create_example_16_4()
x_minimizer, iter_count = minimize_quadratic_problem(problem)
final_printout(problem.x0, problem.solution, x_minimizer, iter_count, problem.calc_f_at, problem.calc_gradient_at)

Initial x is :		None
Optimal x is :		[1.4 1.7]
Approximated x is :	[1.4 1.7]
Is close verification: 	[ True  True]

Function value in optimal point:	-6.450000000000001
Function value in approximated point:   -6.450000000000001
Is close verification:	True

Gradient approximation in optimal point is:
[ 0.80000001 -1.59999999]

Gradient approximation in approximated point is:
[ 0.80000001 -1.59999999]

Is close verification:
[ True  True]

Number of iterations required: 3


# Problem 3
Taken from http://lendek.net/teaching/OPT/qp.pdf (example 1.4)

\begin{align*}
\min x_1^2 + x_2^2 - 4x_1 - 4x_2 &\text{ subject to} \\
x_1 + x_2 &\leq 2 \\
x_1 - 2x_2 + 6 &\leq 2 \\
-x_1 - x_2 &\leq 1 \\
-2x_1 + x_2 &\leq 2 \\
\end{align*}

In [5]:
problem = create_another_example()
x_minimizer, iter_count = minimize_quadratic_problem(problem, use_nullspace_method=True)
final_printout(problem.x0, problem.solution, x_minimizer, iter_count, problem.calc_f_at, problem.calc_gradient_at)

Initial x is :		None
Optimal x is :		[1. 1.]
Approximated x is :	[1. 1.]
Is close verification: 	[ True  True]

Function value in optimal point:	-6.0
Function value in approximated point:   -6.0
Is close verification:	True

Gradient approximation in optimal point is:
[-2. -2.]

Gradient approximation in approximated point is:
[-2. -2.]

Is close verification:
[ True  True]

Number of iterations required: 3
